In [8]:
# As usual, a bit of setup
from __future__ import print_function
import time, os, json
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from cs231n.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from cs231n.rnn_layers import *
from cs231n.captioning_solver import CaptioningSolver
from cs231n.classifiers.rnn import CaptioningRNN
from cs231n.coco_utils import load_coco_data, sample_coco_minibatch, decode_captions, sample_coco_minibatch_custom
from cs231n.image_utils import image_from_url
from cs231n.CaptioningModel import CaptioningModel

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
def BLEU_score(gt_caption, sample_caption):
    """
    gt_caption: string, ground-truth caption
    sample_caption: string, your model's predicted caption
    Returns unigram BLEU score.
    """
    import nltk
    reference = [x for x in gt_caption.split(' ') 
                 if ('<END>' not in x and '<START>' not in x and '<UNK>' not in x)]
    hypothesis = [x for x in sample_caption.split(' ') 
                  if ('<END>' not in x and '<START>' not in x and '<UNK>' not in x)]
    BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis, weights = [1])
    return BLEUscore

In [10]:
# Load COCO data from disk; this returns a dictionary
# We'll work with dimensionality-reduced features for this notebook, but feel
# free to experiment with the original features by changing the flag below.
data = load_coco_data(pca_features=True)

# Print out all the keys and values from the data dictionary
for k, v in data.items():
    if type(v) == np.ndarray:
        print(k, type(v), v.shape, v.dtype)
    else:
        print(k, type(v), len(v))

train_image_idxs <class 'numpy.ndarray'> (400135,) int32
word_to_idx <class 'dict'> 1004
train_features <class 'numpy.ndarray'> (82783, 512) float32
val_features <class 'numpy.ndarray'> (40504, 512) float32
train_urls <class 'numpy.ndarray'> (82783,) <U63
val_captions <class 'numpy.ndarray'> (195954, 17) int32
train_captions <class 'numpy.ndarray'> (400135, 17) int32
val_urls <class 'numpy.ndarray'> (40504,) <U63
idx_to_word <class 'list'> 1004
val_image_idxs <class 'numpy.ndarray'> (195954,) int32


In [11]:
# np.random.seed(231)
# small_data = load_coco_data(max_train=50)
# sample_data = sample_coco_minibatch_custom(small_data, batch_size=50)

In [12]:
# capt_placeholder = tf.placeholder(tf.int32, [None, 17], name='capt_placeholder')
# image_idxs_placeholder = tf.placeholder(tf.int32, [None], name='image_idxs_placeholder')
# features_placeholder = tf.placeholder(tf.float32, data['train_features'].shape, name='features_placeholder')
image_idx = tf.placeholder(tf.int32, shape=data['train_image_idxs'].shape)
train_captions = tf.placeholder(tf.int32, shape=data['train_captions'].shape)
train_features = tf.placeholder(tf.float32, shape=data['train_features'].shape)

def _get_image_features(idxs, capts):
    features = train_features[idxs]
    return features, capts

dataset = tf.data.Dataset.from_tensor_slices((image_idx, train_captions))
dataset = dataset.map(_get_image_features)
dataset = dataset.batch(128)
iterator = dataset.make_initializable_iterator()
next_batch = iterator.get_next();
features, captions = next_batch

In [13]:
model = CaptioningModel(data['word_to_idx'], n_time_steps=16)
loss = model.build_model()
tf.get_variable_scope().reuse_variables()
sample = model.build_sampler()

In [14]:
with tf.variable_scope('training', reuse=tf.AUTO_REUSE):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        train_step = optimizer.minimize(loss)

In [15]:
with tf.Session() as sess:
    # Variables initializer
    sess.run(tf.global_variables_initializer())
    
    feed_dict = {
        image_idx: data['train_image_idxs'],
        train_captions: data['train_captions'],
        train_features: data['train_features']
    }   
    
    # c = sess.run(sample, feed_dict={model.features:data['train_features'][sample_data[1]], model.captions: sample_data[0]})    
    # print(decode_captions(c[0], data['idx_to_word']))
    
    count = 0
    for i in range(5):
        print('Running epoch: %d' % i)
        
        sess.run(iterator.initializer, feed_dict=feed_dict)
        while True:
            try:
                f, c = sess.run([features, captions])
                _, l = sess.run([train_step, loss], feed_dict={model.features:f, model.captions: c})
                count += 1
                if count % 1000 == 0:
                    print('loss: %f' % l)
                    count = 0
            except tf.errors.OutOfRangeError:
                break;
                
    # c = sess.run(sample, feed_dict={model.features:data['train_features'][sample_data[1]], model.captions: sample_data[0]})    
    # print(decode_captions(c[0], data['idx_to_word']))
    
    c = sess.run(sample, feed_dict={model.features:data['val_features'][data['val_image_idxs'][:128]]})
    _c = sess.run(sample, feed_dict={model.features:data['train_features'][data['train_image_idxs'][:128]]})

Running epoch: 0


InternalError: cuDNN launch failure : input shape ([128,1,1,512])
	 [[Node: conv_featuresbatch_norm_2/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NHWC", epsilon=0.001, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv_featuresbatch_norm_2/Reshape, conv_featuresbatch_norm/gamma/read, conv_featuresbatch_norm/beta/read, conv_featuresbatch_norm_2/Const, conv_featuresbatch_norm_2/Const)]]
	 [[Node: truediv_1/_49 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_5499_truediv_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'conv_featuresbatch_norm_2/FusedBatchNorm', defined at:
  File "c:\program files\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\program files\python35\lib\runpy.py", line 85, in _run_code
 

In [14]:
for gt_caption, sample_caption in zip(decode_captions(data['train_captions'][data['train_image_idxs'][:100]], data['idx_to_word']), 
                                      decode_captions(_c, data['idx_to_word'])):
    print('Gt: %s' % gt_caption)
    print('Sample: %s' % sample_caption)

Gt: <START> a little boy wearing sunglasses laying on a shelf in a <UNK> <END>
Sample: a white refrigerator <UNK> sitting in a kitchen <END>
Gt: <START> man on bike <UNK> green vehicle on road <END>
Sample: a wooden <UNK> <UNK> with a <UNK> <UNK> <UNK> <END>
Gt: <START> a little boy wearing sunglasses laying on a shelf in a <UNK> <END>
Sample: a white refrigerator <UNK> sitting in a kitchen <END>
Gt: <START> man on bike <UNK> green vehicle on road <END>
Sample: a wooden <UNK> <UNK> with a <UNK> <UNK> <UNK> <END>
Gt: <START> two old toilets and an old sink in a small room <END>
Sample: a clock on a pole near a tree <END>
Gt: <START> two old toilets and an old sink in a small room <END>
Sample: a clock on a pole near a tree <END>
Gt: <START> a little boy wearing sunglasses laying on a shelf in a <UNK> <END>
Sample: a white refrigerator <UNK> sitting in a kitchen <END>
Gt: <START> a little boy wearing sunglasses laying on a shelf in a <UNK> <END>
Sample: a white refrigerator <UNK> sitting

In [12]:
total_score = 0.0
for gt_caption, sample_caption in zip(decode_captions(data['val_captions'][:1000], data['idx_to_word']), 
                                      decode_captions(c, data['idx_to_word'])):
    total_score += BLEU_score(gt_caption, sample_caption)
print('BLEU score: %f' % (total_score/len(c)))

BLEU score: 0.201597


In [10]:
with tf.Session() as sess:
    feed_dict = {
        model.captions: sample_data[0],
        model.image_idxs: sample_data[1],
        model.features: data['train_features'][sample_data[1]]
    }
    sess.run(tf.global_variables_initializer())
    c = sess.run(sample, feed_dict=feed_dict)
    print(decode_captions(c[:2], data['idx_to_word']))
    for i in range(50):
        # sess.run(iterator.initializer, feed_dict=feed_dict)
        # while True:
            try:
                #_, l = sess.run([train_step, loss], feed_dict=feed_dict)
                
                #if i % 10 == 0: print(l)
                _, l = sess.run([train_step, loss], feed_dict=feed_dict)
            except tf.errors.OutOfRangeError:
                break;
                
    c = sess.run(sample, feed_dict=feed_dict)
    print(decode_captions(sample_data[0][:2], data['idx_to_word']))
    print(decode_captions(c[:2], data['idx_to_word']))

['feet event scissors lone about stacked party party party plastic rest wire wire wire lying traveling candles candles pen computer', 'prepared dining prepared dining using is park bicycle ice lettuce mirror game himself himself square rain himself swings pink catch']
['<START> half a <UNK> <UNK> out on the ocean <END>', '<START> a man standing on the side of a road with bags of luggage <END>']
['half a <UNK> <UNK> out on the ocean <END>', 'a man standing on the side of a road with bags of luggage <END>']
